In [2]:
from pathlib import Path
import os
from tqdm import tqdm
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from fastai.data.all import get_image_files
# from fastai.vision.all import *
from PIL import Image
import numpy as np
import joblib

In [9]:
import inspect
import sys

currentdir = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.insert(0, currentdir) 

In [10]:
from src.utils import load_images_recursively

In [16]:
image_file_extensions = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def is_image_path_valid(path: Path):
    return path.is_file() and path.suffix in image_file_extensions

def load_image_file(path):
    return Image.open(path)

image_dir = Path("../data/before_after_images/processed")

def load_image_paths(path: Path):
    fns = get_image_files(path)
    
    return fns

before_paths = load_image_paths(image_dir / 'before')
after_paths = load_image_paths(image_dir / 'after')

mean_rgb = (131.0912, 103.8827, 91.4953)
image_shape = (224,224,3)

def load_image_for_feature_extraction(path='', shape=image_shape):
    '''
    Referenced from VGGFace2 Paper:
    Q. Cao, L. Shen, W. Xie, O. M. Parkhi, and A. Zisserman, “VGGFace2: A dataset for recognising faces across pose and age,” arXiv:1710.08092 [cs], May 2018
    '''
    short_size = 224.0
    crop_size = shape
    img = Image.open(path)
    im_shape = np.array(img.size)    # in the format of (width, height, *)
    img = img.convert('RGB')

    ratio = float(short_size) / np.min(im_shape)
    img = img.resize(size=(int(np.ceil(im_shape[0] * ratio)),   # width
                           int(np.ceil(im_shape[1] * ratio))),  # height
                     resample=Image.BILINEAR)

    x = np.array(img)  # image has been transposed into (height, width)
    newshape = x.shape[:2]
    h_start = (newshape[0] - crop_size[0])//2
    w_start = (newshape[1] - crop_size[1])//2
    x = x[h_start:h_start+crop_size[0], w_start:w_start+crop_size[1]]
    
    # normalize colors to prevent overfitting on color differences 
    x = x - mean_rgb
    
    # returns transformed image, and original image
    return x

In [13]:
from saved_model.prepare_resnet50 import prepare_resnet_model

resnet_model = prepare_resnet_model("../saved_model/resnet50_ft_weight.pkl")

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def full_pipeline(x):
    x = torch.Tensor(x.transpose(0, 3, 1, 2))  # nx3x224x224
    x = x.to(device)
    x = resnet_model(x).detach().cpu()
    return x

In [15]:
def preprocess(path):
    return np.expand_dims(load_image_for_feature_extraction(path), 0)

In [33]:
before_features = torch.empty((len(before_paths), 2048))
after_features = torch.empty((len(after_paths), 2048))

In [35]:
for i, (before_path, after_path) in tqdm(enumerate(zip(before_paths, after_paths)), total=len(before_paths)):
    if before_path.stem != after_path.stem:
        print(f"Before and after don't match for index {i}, before: {before_path}, after: {after_path}")
        break
        
    before_features[i] = full_pipeline(preprocess(before_path)).squeeze()
    after_features[i] = full_pipeline(preprocess(after_path)).squeeze()

100%|███████████████████████████████████████| 1292/1292 [00:25<00:00, 50.04it/s]


In [38]:
before_features[-1]

tensor([0.1049, 0.0000, 2.3196,  ..., 2.6292, 2.7984, 0.0230])

In [39]:
torch.save(before_features, "../data/before_after_images/features/resnet_50_vggface2/before/list_of_tensors.pt")

In [40]:
torch.save(after_features, "../data/before_after_images/features/resnet_50_vggface2/after/list_of_tensors.pt")